# Doodle classifier


In [2]:
# Setup
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import os
import sys
root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

from src.data_loader import DataLoader as DL

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Training

In [3]:
dl = DL()
data, labels = dl.load_data_batches(batch_size=10_000)
data_1d = np.array([dl.one_dimensional_array_from_matrix(mat) for mat in data])
X_train, X_test, y_train, y_test = train_test_split(data_1d, labels, test_size=0.3, shuffle=True)

In [3]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

21000 9000 21000 9000


In [4]:
# hidden_layer_sizes=(128, 128, 64)
clf = MLPClassifier(hidden_layer_sizes=(128,))
clf.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(128,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

## Prediction

In [5]:
clf.score(X_test, y_test)

0.7733333333333333

### Results

| Parameter | Value | Results (Accuray) |
| --- | --- | --- |
| Hidden Layers | `(100,)` (default) | `0.811` |
| Hidden Layers | `(128, 128, 64)` | `0.868` |
| Hidden Layers | `(128,)` | `0.863` |


